并行(Parallelism)是并发(concurrency)的特例，所有并行的系统都是并发的，但不是所有的并发系统是并行的。

为了并行执行200个任务，你需要200个CPU。所以实际上，大多数计算是并发但不是并行的。



进程、线程和Python的GIL
1. 每个Python解释器的实例都是一个进程。你可以使用`multiprocessing`或`concurrent.futures`包开启额外的Python进程。
2. Python解释器使用单个线程来运行用户程序和内存垃圾收集器。你可以使用`threading`或`concurrent.futures`包启动额外的Python线程。
3. 对对象引用计数和其他内部解释器状态的访问由一个锁控制，即全局解释器锁（GIL,Global Interpreter Lock）。只有一个Python线程可以持有GIL。这意味着任何时刻只能有一个线程运行Python代码，无论CPU核数有多少。
4. 为了防止一个Python线程无限期地持有GIL，Python的字节码解释器默认每5ms暂停当前的Python线程，释放GIL。该线程也可以继续尝试申请GIL，但不一定成功(有其他线程等待GIL的情况下)。
5. 我们无法控制GIL。但一些内置函数或C编写的扩展可以在运行耗时任务时释放GIL。
6. 每个进行系统调用(syscall)的Python标准库函数都会释放GIL。包括所有执行磁盘I/O、网络I/O的函数以及`time.sleep()`。
7. 在Python/C API级别集成的扩展还可以启动其他不受GIL影响的非Python线程。尽管这样的线程不能改变Python对象，但能读写支持缓冲协议(buffer protocol)的底层对象的内存，比如`bytearray`、`array.array`和`Numpy`数组。
8. GIL对使用Python线程的网络编程的影响相对较小，因为I/O函数会释放GIL，而读写网络总是意味着高延迟——与读写内存相比。
9. 为了在多核上运行CPU密集的Python代码，你必须使用多个Python进程。